In [0]:
#!/usr/bin/python3

import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import glob
import csv
import time
import matplotlib.pyplot as plt
import math

testInputFile = "./testInput.csv"
testTargetFile = "./testTarget.csv"

trainInputFiles = sorted(glob.glob("./trainInput*.csv"))
trainTargetFiles = sorted(glob.glob("./trainTarget*.csv"))

timeArray = []
averageArrayGaussian = []
averageArrayPoly = []
degrees = range(1, 5)
sigmas = range(1, 7)
def readInData(file, arrayToPopulate):
    ''' Parameters:
        dataPath: CSV file to extract from
        arrayToPopulate: Array to add values to'''
    with open(file, "r") as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            arrayToPopulate.append(row)

def listToNumpyList(list):
    for entry in list:
        entry = np.array(entry)
    return list

def sort(input):
	return input[0]

def crossGaussian():
    sigmasMax = [] #degreeWeight average
    for sigma in sigmas:
        start = time.time()
        accuracyAverage = []
        for j in range(10):  # 10-fold validation
            testInput = []
            testTarget = []
            trainInput = []
            trainTarget = []

            readInData(trainInputFiles[j], testInput)  # Read in one train data set as test data
            readInData(trainTargetFiles[j], testTarget)  # Read in one train label set as test label
            for l in range(0, j):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)
            for l in range(j + 1, 10):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)
            gaussianMean = gaussianKernel(trainInput, trainTarget, testInput, testTarget, sigma)
            accuracyAverage.append(gaussianMean)
        averageArrayGaussian.append(np.mean(accuracyAverage))
        sigmasMax.append([np.mean(accuracyAverage), sigma])
        end = time.time()
        timeArray.append(end - start)
    bestSigma = min(sigmasMax, key=sort)[1]
    print("Best sigma:")
    print(bestSigma)
    print("The loss of the best sigma on test set:")
    print(bestGaussian(bestSigma))

def gaussianKernel(trainInput, trainTarget, testInput, testTarget, sigma):
    K = []
    for i in range(len(trainInput)):
        trainInputI = np.array(trainInput[i]).astype('float')
        for j in range(len(trainInput)):
            trainInputJ = np.array(trainInput[j]).astype('float')
            exponent = np.matmul((trainInputI - trainInputJ).transpose(), trainInputI - trainInputJ)
            exponent /= -(2*sigma*sigma)
            K.append(np.array(np.exp(exponent)))
    K = np.array(K).reshape(len(trainInput), len(trainInput))
    trainingUse = np.matmul(np.linalg.inv((K + np.identity(len(K)))), np.array(trainTarget).astype('float'))
    loss = 0
    for i in range(len(testInput)):
        testInputNp = np.array(testInput[i]).astype('float')
        kernel = []
        for j in range(len(trainInput)):
            trainInputNp = np.array(trainInput[j]).astype('float')
            exponent = np.matmul((testInputNp - trainInputNp).transpose(), testInputNp - trainInputNp)
            exponent /= -(2*sigma*sigma)
            kernel.append(np.exp(exponent))
        f = np.matmul(np.array(kernel), trainingUse)  # y*
        difference = (float)(testTarget[i][0]) - f
        loss = loss + (difference * difference)
    return 0.5 * loss

def bestGaussian(bestSigma):
    testInput = []
    testTarget = []
    trainInput = []
    trainTarget = []

    readInData(testInputFile, testInput)
    readInData(testTargetFile, testTarget)
    for inputFile in trainInputFiles:
        readInData(inputFile, trainInput)
    for targetFile in trainTargetFiles:
        readInData(targetFile, trainTarget)
    return gaussianKernel(trainInput, trainTarget, testInput, testTarget, bestSigma)

def crossPoly():
    degreesMax = []  # degreeWeight average
    for d in degrees:
        start = time.time()
        accuracyAverage = []
        for j in range(10):  # 10-fold validation
            testInput = []
            testTarget = []
            trainInput = []
            trainTarget = []

            readInData(trainInputFiles[j], testInput)  # Read in one train data set as test data
            readInData(trainTargetFiles[j], testTarget)  # Read in one train label set as test label
            for l in range(0, j):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)
            for l in range(j + 1, 10):
                readInData(trainInputFiles[l], trainInput)
                readInData(trainTargetFiles[l], trainTarget)
            polyMean = polyKernel(trainInput, trainTarget, testInput, testTarget, d)
            accuracyAverage.append(polyMean)
        averageArrayPoly.append(np.mean(accuracyAverage))
        degreesMax.append([np.mean(accuracyAverage), d])
        end = time.time()
        timeArray.append(end - start)
    bestDegree = min(degreesMax, key=sort)[1]
    print("Best degree:")
    print(bestDegree)
    print("The loss of the best degree on test set:")
    print(bestPoly(bestDegree))

def polyKernel(trainInput, trainTarget, testInput, testTarget, d):
    K = []
    for i in range(len(trainInput)):
        trainInputI = np.array(trainInput[i]).astype('float')
        for j in range(len(trainInput)):
            trainInputJ = np.array(trainInput[j]).astype('float')
            polynomial = (np.matmul(trainInputI, trainInputJ) + 1) ** d
            K.append(polynomial)
    K = np.array(K).reshape(len(trainInput), len(trainInput))

    trainingUse = np.matmul(np.linalg.inv((K + np.identity(len(K)))), np.array(trainTarget).astype('float'))
    loss = 0
    for i in range(len(testInput)):
        testInputNp = np.array(testInput[i]).astype('float')
        kernel = []
        for j in range(len(trainInput)):
            trainInputNp = np.array(trainInput[j]).astype('float')
            polynomial = (np.matmul(testInputNp, trainInputNp) + 1) ** d
            kernel.append(polynomial)
        f = np.matmul(np.array(kernel), trainingUse)  # y*
        difference = (float)(testTarget[i][0]) - f
        loss = loss + (difference * difference)
    return 0.5 * loss

def bestPoly(bestDegree):
    testInput = []
    testTarget = []
    trainInput = []
    trainTarget = []

    readInData(testInputFile, testInput)
    readInData(testTargetFile, testTarget)
    for inputFile in trainInputFiles:
        readInData(inputFile, trainInput)
    for targetFile in trainTargetFiles:
        readInData(targetFile, trainTarget)
    return polyKernel(trainInput, trainTarget, testInput, testTarget, bestDegree)


def identityKernel():
    testInput = []
    testTarget = []
    trainInput = []
    trainTarget = []

    readInData(testInputFile, testInput)
    readInData(testTargetFile, testTarget)
    for inputFile in trainInputFiles:
        readInData(inputFile, trainInput)
    for targetFile in trainTargetFiles:
        readInData(targetFile, trainTarget)

    phi = np.array(trainInput).astype('float')
    K = np.matmul(phi, phi.transpose())

    trainingUse = np.matmul(np.linalg.inv((K + np.identity(len(K)))), np.array(trainTarget).astype('float')) 
    loss = 0
    for i in range(len(testInput)):
        kernel = np.matmul(np.array(testInput[i]).astype('float'),
                           np.array(trainInput).astype('float').transpose()) # kernel identity
        f = np.matmul(kernel.reshape(1,200), trainingUse) # y*
        difference = (float)(testTarget[i][0]) - f
        loss = loss + (difference * difference)
    return 0.5 * loss

crossPoly()
plt.plot(degrees, averageArrayPoly)
plt.savefig("q2cPolyError.png")
